# package


In [1]:
import json
import math
import time

import mygene
import numpy as np
import pandas as pd
import requests
import sympy as sym
from flatsplode import flatsplode

# geneinfo


## human


In [2]:
# input_symbol = "PTH1R"
input_symbol = "IRF7"
# input_symbol = "T"
# input_symbol = ""

In [3]:
mg = mygene.MyGeneInfo()

In [4]:
genetable0 = mg.query(
    "symbol:" + input_symbol,
    fields="symbol, name, ensembl.gene, entrezgene, uniprot.Swiss-Prot, homologene.id",
    species="human",
)

In [5]:
genetable0

{'took': 3,
 'total': 1,
 'max_score': 17.752981,
 'hits': [{'_id': '3665',
   '_score': 17.752981,
   'ensembl': [{'gene': 'ENSG00000276561'}, {'gene': 'ENSG00000185507'}],
   'entrezgene': '3665',
   'homologene': {'id': 128624},
   'name': 'interferon regulatory factor 7',
   'symbol': 'IRF7',
   'uniprot': {'Swiss-Prot': 'Q92985'}}]}

In [6]:
pd.DataFrame(list(flatsplode(genetable0["hits"][0])))

,_id,_score,ensembl.gene,entrezgene,homologene.id,name,symbol,uniprot.Swiss-Prot
0,3665,17.752981,ENSG00000276561,3665,128624,interferon regulatory factor 7,IRF7,Q92985
1,3665,17.752981,ENSG00000185507,3665,128624,interferon regulatory factor 7,IRF7,Q92985


In [7]:
if genetable0["total"] == 0:
    print("Please input the correct gene symbol.")

In [8]:
genetable00 = pd.DataFrame(
    {
        "symbol": pd.Series([], dtype=object),
        "name": pd.Series([], dtype=object),
        "ensembl.gene": pd.Series([], dtype=object),
        "entrezgene": pd.Series([], dtype=object),
        "uniprot.Swiss-Prot": pd.Series([], dtype=object),
        "homologene.id": pd.Series([], dtype=np.int64),
    }
)

In [9]:
display(genetable00)

display(genetable00.shape)

display(type(genetable00))

,symbol,name,ensembl.gene,entrezgene,uniprot.Swiss-Prot,homologene.id


(0, 6)

pandas.core.frame.DataFrame

In [10]:
genetable = (
    pd.concat([genetable00, pd.DataFrame(list(flatsplode(genetable0["hits"][0])))])
    .filter(
        [
            "symbol",
            "name",
            "ensembl.gene",
            "entrezgene",
            "uniprot.Swiss-Prot",
            "homologene.id",
        ]
    )
    .rename(
        columns={
            "ensembl.gene": "ensembl_gene_ID",
            "entrezgene": "entrezgene_ID",
            "uniprot.Swiss-Prot": "uniprot_ID",
            "homologene.id": "homologene_ID",
        }
    )
)

In [11]:
display(genetable)
display(genetable.shape)
display(type(genetable))

,symbol,name,ensembl_gene_ID,entrezgene_ID,uniprot_ID,homologene_ID
0,IRF7,interferon regulatory factor 7,ENSG00000276561,3665,Q92985,128624
1,IRF7,interferon regulatory factor 7,ENSG00000185507,3665,Q92985,128624


(2, 6)

pandas.core.frame.DataFrame

In [12]:
gene_ensembls = genetable.ensembl_gene_ID.tolist()

In [13]:
display(gene_ensembls)
display(len(gene_ensembls))
display(type(gene_ensembls))

['ENSG00000276561', 'ENSG00000185507']

2

list

In [14]:
base_url = "https://api.platform.opentargets.org/api/v4/graphql"

In [15]:
query_geneinfo = """
    query target ($ensemblId: String!) {
        target (ensemblId: $ensemblId) {
            id
            approvedSymbol
            approvedName
        }
    }
"""

In [16]:
for gene_ensembl in gene_ensembls:
    variables0 = {"ensemblId": gene_ensembl}

    r0 = requests.post(
        base_url, json={"query": query_geneinfo, "variables": variables0}
    )

    res0 = json.loads(r0.text)

    if res0["data"]["target"] == None:
        continue
    else:
        break

In [17]:
ensembl_gene_ok = pd.DataFrame(list(flatsplode(res0["data"]["target"])))["id"][0]

In [18]:
display(ensembl_gene_ok)
display(len(ensembl_gene_ok))
display(type(ensembl_gene_ok))

'ENSG00000185507'

15

str

In [19]:
genetable_ok = genetable.query("ensembl_gene_ID == @ensembl_gene_ok").reset_index(
    drop=True
)

In [20]:
display(genetable_ok)
display(genetable_ok.shape)
display(type(genetable_ok))

,symbol,name,ensembl_gene_ID,entrezgene_ID,uniprot_ID,homologene_ID
0,IRF7,interferon regulatory factor 7,ENSG00000185507,3665,Q92985,128624


(1, 6)

pandas.core.frame.DataFrame

In [21]:
# input_symbol = "PTH1R"
input_symbol = "IRF7"
# input_symbol = "T"
# input_symbol = ""

mg = mygene.MyGeneInfo()

genetable0 = mg.query(
    "symbol:" + input_symbol,
    fields="symbol, name, ensembl.gene, entrezgene, uniprot.Swiss-Prot, homologene.id",
    species="human",
)

if genetable0["total"] == 0:
    print("Please input the correct gene symbol.")
else:
    genetable00 = pd.DataFrame(
        {
            "symbol": pd.Series([], dtype=object),
            "name": pd.Series([], dtype=object),
            "ensembl.gene": pd.Series([], dtype=object),
            "entrezgene": pd.Series([], dtype=object),
            "uniprot.Swiss-Prot": pd.Series([], dtype=object),
            "homologene.id": pd.Series([], dtype=np.int64),
        }
    )

    genetable = (
        pd.concat([genetable00, pd.DataFrame(list(flatsplode(genetable0["hits"][0])))])
        .filter(
            [
                "symbol",
                "name",
                "ensembl.gene",
                "entrezgene",
                "uniprot.Swiss-Prot",
                "homologene.id",
            ]
        )
        .rename(
            columns={
                "ensembl.gene": "ensembl_gene_ID",
                "entrezgene": "entrezgene_ID",
                "uniprot.Swiss-Prot": "uniprot_ID",
                "homologene.id": "homologene_ID",
            }
        )
    )

    gene_ensembls = genetable.ensembl_gene_ID.tolist()

    base_url = "https://api.platform.opentargets.org/api/v4/graphql"

    query_geneinfo = """
        query target ($ensemblId: String!) {
            target (ensemblId: $ensemblId) {
                id
                approvedSymbol
                approvedName
            }
        }
    """

    res0 = {}

    for gene_ensembl in gene_ensembls:
        variables0 = {"ensemblId": gene_ensembl}

        r0 = requests.post(
            base_url, json={"query": query_geneinfo, "variables": variables0}
        )

        res0 = json.loads(r0.text)

        if res0["data"]["target"] == None:
            continue
        else:
            break

    ensembl_gene_ok = pd.DataFrame(list(flatsplode(res0["data"]["target"])))["id"][0]

    genetable_ok = genetable.query("ensembl_gene_ID == @ensembl_gene_ok").reset_index(
        drop=True
    )

In [22]:
ensembl_gene_ok

'ENSG00000185507'

In [23]:
display(genetable_ok)
display(genetable_ok.shape)
display(type(genetable_ok))

,symbol,name,ensembl_gene_ID,entrezgene_ID,uniprot_ID,homologene_ID
0,IRF7,interferon regulatory factor 7,ENSG00000185507,3665,Q92985,128624


(1, 6)

pandas.core.frame.DataFrame

## homologene


In [24]:
display(genetable_ok.homologene_ID.tolist()[0])
display(len(genetable_ok.homologene_ID.tolist()))
display(type(genetable_ok.homologene_ID.tolist()[0]))
display(np.isnan(genetable_ok.homologene_ID.tolist()[0]))

128624

1

int

False

### mouse


In [25]:
genetable_m0 = mg.query(
    "homologene.id: " + str(genetable_ok.homologene_ID.tolist()[0]),
    fields="symbol, name, ensembl.gene, entrezgene, uniprot.Swiss-Prot, homologene.id",
    species="mouse",
)

In [26]:
genetable_m0

{'took': 5,
 'total': 1,
 'max_score': 1.3,
 'hits': [{'_id': '54123',
   '_score': 1.3,
   'ensembl': {'gene': 'ENSMUSG00000025498'},
   'entrezgene': '54123',
   'homologene': {'id': 128624},
   'name': 'interferon regulatory factor 7',
   'symbol': 'Irf7',
   'uniprot': {'Swiss-Prot': 'P70434'}}]}

In [27]:
pd.DataFrame(list(flatsplode(genetable_m0["hits"][0])))

,_id,_score,ensembl.gene,entrezgene,homologene.id,name,symbol,uniprot.Swiss-Prot
0,54123,1.3,ENSMUSG00000025498,54123,128624,interferon regulatory factor 7,Irf7,P70434


In [28]:
genetable_m0["total"] == 0

False

In [29]:
genetable_m = (
    pd.concat([genetable00, pd.DataFrame(list(flatsplode(genetable_m0["hits"][0])))])
    .filter(
        [
            "symbol",
            "name",
            "ensembl.gene",
            "entrezgene",
            "uniprot.Swiss-Prot",
            "homologene.id",
        ]
    )
    .rename(
        columns={
            "ensembl.gene": "ensembl_gene_ID",
            "entrezgene": "entrezgene_ID",
            "uniprot.Swiss-Prot": "uniprot_ID",
            "homologene.id": "homologene_ID",
        }
    )
)

In [30]:
display(genetable_m)
display(genetable_m.shape)
display(type(genetable_m))

,symbol,name,ensembl_gene_ID,entrezgene_ID,uniprot_ID,homologene_ID
0,Irf7,interferon regulatory factor 7,ENSMUSG00000025498,54123,P70434,128624


(1, 6)

pandas.core.frame.DataFrame

In [31]:
genetable_m0 = mg.query(
    "homologene.id: " + str(genetable_ok.homologene_ID.tolist()[0]),
    fields="symbol, name, ensembl.gene, entrezgene, uniprot.Swiss-Prot, homologene.id",
    species="mouse",
)

genetable_m = (
    pd.concat([genetable00, pd.DataFrame(list(flatsplode(genetable_m0["hits"][0])))])
    .filter(
        [
            "symbol",
            "name",
            "ensembl.gene",
            "entrezgene",
            "uniprot.Swiss-Prot",
            "homologene.id",
        ]
    )
    .rename(
        columns={
            "ensembl.gene": "ensembl_gene_ID",
            "entrezgene": "entrezgene_ID",
            "uniprot.Swiss-Prot": "uniprot_ID",
            "homologene.id": "homologene_ID",
        }
    )
)

In [32]:
display(genetable_m)
display(genetable_m.shape)
display(type(genetable_m))

,symbol,name,ensembl_gene_ID,entrezgene_ID,uniprot_ID,homologene_ID
0,Irf7,interferon regulatory factor 7,ENSMUSG00000025498,54123,P70434,128624


(1, 6)

pandas.core.frame.DataFrame

### rat


In [33]:
genetable_r0 = mg.query(
    "homologene.id: " + str(genetable_ok.homologene_ID.tolist()[0]),
    fields="symbol, name, ensembl.gene, entrezgene, uniprot.Swiss-Prot, homologene.id",
    species="rat",
)

In [34]:
genetable_r0

{'took': 3,
 'total': 1,
 'max_score': 1.1,
 'hits': [{'_id': '293624',
   '_score': 1.1,
   'ensembl': {'gene': 'ENSRNOG00000017414'},
   'entrezgene': '293624',
   'homologene': {'id': 128624},
   'name': 'interferon regulatory factor 7',
   'symbol': 'Irf7'}]}

In [35]:
pd.DataFrame(list(flatsplode(genetable_r0["hits"][0])))

,_id,_score,ensembl.gene,entrezgene,homologene.id,name,symbol
0,293624,1.1,ENSRNOG00000017414,293624,128624,interferon regulatory factor 7,Irf7


In [36]:
genetable_r0["total"] == 0

False

In [37]:
genetable_r = (
    pd.concat([genetable00, pd.DataFrame(list(flatsplode(genetable_r0["hits"][0])))])
    .filter(
        [
            "symbol",
            "name",
            "ensembl.gene",
            "entrezgene",
            "uniprot.Swiss-Prot",
            "homologene.id",
        ]
    )
    .rename(
        columns={
            "ensembl.gene": "ensembl_gene_ID",
            "entrezgene": "entrezgene_ID",
            "uniprot.Swiss-Prot": "uniprot_ID",
            "homologene.id": "homologene_ID",
        }
    )
)

In [38]:
display(genetable_r)
display(genetable_r.shape)
display(type(genetable_r))

,symbol,name,ensembl_gene_ID,entrezgene_ID,uniprot_ID,homologene_ID
0,Irf7,interferon regulatory factor 7,ENSRNOG00000017414,293624,NaN,128624


(1, 6)

pandas.core.frame.DataFrame

In [39]:
genetable_r0 = mg.query(
    "homologene.id: " + str(genetable_ok.homologene_ID.tolist()[0]),
    fields="symbol, name, ensembl.gene, entrezgene, uniprot.Swiss-Prot, homologene.id",
    species="rat",
)

genetable_r = (
    pd.concat([genetable00, pd.DataFrame(list(flatsplode(genetable_r0["hits"][0])))])
    .filter(
        [
            "symbol",
            "name",
            "ensembl.gene",
            "entrezgene",
            "uniprot.Swiss-Prot",
            "homologene.id",
        ]
    )
    .rename(
        columns={
            "ensembl.gene": "ensembl_gene_ID",
            "entrezgene": "entrezgene_ID",
            "uniprot.Swiss-Prot": "uniprot_ID",
            "homologene.id": "homologene_ID",
        }
    )
)

In [40]:
display(genetable_r)
display(genetable_r.shape)
display(type(genetable_r))

,symbol,name,ensembl_gene_ID,entrezgene_ID,uniprot_ID,homologene_ID
0,Irf7,interferon regulatory factor 7,ENSRNOG00000017414,293624,NaN,128624


(1, 6)

pandas.core.frame.DataFrame

# score


## disease size


In [41]:
query_associatedDiseases_info = """
    query target ($ensemblId: String!) {
        target (ensemblId: $ensemblId) {
            associatedDiseases (enableIndirect: false) {
                count
            }
        }
    }
"""

In [42]:
variables01 = {"ensemblId": ensembl_gene_ok}

In [43]:
r01 = requests.post(
    base_url, json={"query": query_associatedDiseases_info, "variables": variables01}
)

In [44]:
res01 = json.loads(r01.text)

In [45]:
disease_size = res01["data"]["target"]["associatedDiseases"]["count"]

In [46]:
display(disease_size)
display(type(disease_size))

245

int

## overall


In [47]:
query_associatedDiseases_scores = """
    query target ($ensemblId: String!, $size: Int!) {
        target (ensemblId: $ensemblId) {
            associatedDiseases (enableIndirect: false, page: {index: 0, size: $size}) {
                rows {
                    score
                    disease {
                        id
                        name
                    }
                }
            }
        }
    }
"""

In [48]:
variables02 = {"ensemblId": ensembl_gene_ok, "size": disease_size}

In [49]:
r02 = requests.post(
    base_url, json={"query": query_associatedDiseases_scores, "variables": variables02}
)

In [50]:
res02 = json.loads(r02.text)

In [51]:
overall_table = (
    pd.DataFrame(list(flatsplode(res02["data"]["target"]["associatedDiseases"])))
    .filter(["rows.disease.id", "rows.disease.name", "rows.score"])
    .rename(
        columns={
            "rows.disease.id": "disease_ID",
            "rows.disease.name": "disease_name",
            "rows.score": "overall_score",
        }
    )
)

In [52]:
display(overall_table.head())
display(overall_table.shape)
display(type(overall_table))

,disease_ID,disease_name,overall_score
0,MONDO_0015823,primary immunodeficiency due to a defect in ad...,0.482153
1,MONDO_0007915,systemic lupus erythematosus,0.342453
2,EFO_0004705,hypothyroidism,0.218722
3,EFO_0000571,lung adenocarcinoma,0.189006
4,MONDO_0011962,endometrial cancer,0.184790


(245, 3)

pandas.core.frame.DataFrame

## datatype


In [53]:
query_associatedDiseases_scores_dt = """
    query target ($ensemblId: String!, $size: Int!) {
        target (ensemblId: $ensemblId) {
            associatedDiseases (enableIndirect: false, page: {index: 0, size: $size}) {
                rows {
                    datatypeScores {
                        id
                        score
                    }
                    disease {
                        id
                        name
                    }
                }
            }
        }
    }
"""

In [54]:
r03 = requests.post(
    base_url,
    json={"query": query_associatedDiseases_scores_dt, "variables": variables02},
)

In [55]:
res03 = json.loads(r03.text)

In [56]:
dt_table0 = (
    pd.DataFrame(list(flatsplode(res03["data"]["target"]["associatedDiseases"])))
    .filter(
        [
            "rows.disease.id",
            "rows.disease.name",
            "rows.datatypeScores.id",
            "rows.datatypeScores.score",
        ]
    )
    .rename(
        columns={
            "rows.disease.id": "disease_ID",
            "rows.disease.name": "disease_name",
            "rows.datatypeScores.id": "datatype",
            "rows.datatypeScores.score": "datatype_score",
        }
    )
    .set_index(["disease_ID", "disease_name", "datatype"])["datatype_score"]
    .unstack()
    .reset_index()
)

In [57]:
display(dt_table0.head())
display(dt_table0.shape)
display(type(dt_table0))

datatype,disease_ID,disease_name,affected_pathway,animal_model,genetic_association,literature,rna_expression
0,EFO_0000180,HIV-1 infection,NaN,NaN,NaN,0.128679,NaN
1,EFO_0000181,head and neck squamous cell carcinoma,0.330597,NaN,NaN,0.034956,NaN
2,EFO_0000182,hepatocellular carcinoma,NaN,NaN,NaN,0.286361,NaN
3,EFO_0000198,myelodysplastic syndrome,NaN,NaN,NaN,0.018238,NaN
4,EFO_0000199,oral squamous cell carcinoma,NaN,NaN,NaN,0.036476,NaN


(245, 7)

pandas.core.frame.DataFrame

In [58]:
dt_table00 = pd.DataFrame(
    {
        "disease_ID": pd.Series([], dtype=object),
        "disease_name": pd.Series([], dtype=object),
        "known_drug": pd.Series([], dtype=np.float64),
        "genetic_association": pd.Series([], dtype=np.float64),
        "animal_model": pd.Series([], dtype=np.float64),
        "literature": pd.Series([], dtype=np.float64),
        "rna_expression": pd.Series([], dtype=np.float64),
        "affected_pathway": pd.Series([], dtype=np.float64),
        "somatic_mutation": pd.Series([], dtype=np.float64),
    }
)

In [59]:
display(dt_table00)
display(dt_table00.shape)
display(type(dt_table00))

,disease_ID,disease_name,known_drug,genetic_association,animal_model,literature,rna_expression,affected_pathway,somatic_mutation


(0, 9)

pandas.core.frame.DataFrame

In [60]:
dt_table = pd.concat([dt_table00, dt_table0]).fillna(
    {
        "known_drug": 0,
        "genetic_association": 0,
        "animal_model": 0,
        "literature": 0,
        "rna_expression": 0,
        "affected_pathway": 0,
        "somatic_mutation": 0,
    }
)

In [61]:
display(dt_table.head())
display(dt_table.shape)
display(type(dt_table))

,disease_ID,disease_name,known_drug,genetic_association,animal_model,literature,rna_expression,affected_pathway,somatic_mutation
0,EFO_0000180,HIV-1 infection,0.0,0.0,0.0,0.128679,0.0,0.000000,0.0
1,EFO_0000181,head and neck squamous cell carcinoma,0.0,0.0,0.0,0.034956,0.0,0.330597,0.0
2,EFO_0000182,hepatocellular carcinoma,0.0,0.0,0.0,0.286361,0.0,0.000000,0.0
3,EFO_0000198,myelodysplastic syndrome,0.0,0.0,0.0,0.018238,0.0,0.000000,0.0
4,EFO_0000199,oral squamous cell carcinoma,0.0,0.0,0.0,0.036476,0.0,0.000000,0.0


(245, 9)

pandas.core.frame.DataFrame

## datasource


In [62]:
query_associatedDiseases_scores_ds = """
    query target ($ensemblId: String!, $size: Int!) {
        target (ensemblId: $ensemblId) {
            associatedDiseases (enableIndirect: false, page: {index: 0, size: $size}) {
                rows {
                    datasourceScores {
                        id
                        score
                    }
                    disease {
                        id
                        name
                    }
                }
            }
        }
    }
"""

In [63]:
r04 = requests.post(
    base_url,
    json={"query": query_associatedDiseases_scores_ds, "variables": variables02},
)

In [64]:
res04 = json.loads(r04.text)

In [65]:
ds_table0 = (
    pd.DataFrame(list(flatsplode(res04["data"]["target"]["associatedDiseases"])))
    .filter(
        [
            "rows.disease.id",
            "rows.disease.name",
            "rows.datasourceScores.id",
            "rows.datasourceScores.score",
        ]
    )
    .rename(
        columns={
            "rows.disease.id": "disease_ID",
            "rows.disease.name": "disease_name",
            "rows.datasourceScores.id": "datasource",
            "rows.datasourceScores.score": "datasource_score",
        }
    )
    .set_index(["disease_ID", "disease_name", "datasource"])["datasource_score"]
    .unstack()
    .reset_index()
)

In [66]:
display(ds_table0.head())
display(ds_table0.shape)
display(type(ds_table0))

datasource,disease_ID,disease_name,europepmc,eva,expression_atlas,impc,ot_genetics_portal,slapenrich
0,EFO_0000180,HIV-1 infection,0.128679,NaN,NaN,NaN,NaN,NaN
1,EFO_0000181,head and neck squamous cell carcinoma,0.034956,NaN,NaN,NaN,NaN,0.330597
2,EFO_0000182,hepatocellular carcinoma,0.286361,NaN,NaN,NaN,NaN,NaN
3,EFO_0000198,myelodysplastic syndrome,0.018238,NaN,NaN,NaN,NaN,NaN
4,EFO_0000199,oral squamous cell carcinoma,0.036476,NaN,NaN,NaN,NaN,NaN


(245, 8)

pandas.core.frame.DataFrame

In [67]:
ds_table00 = pd.DataFrame(
    {
        "disease_ID": pd.Series([], dtype=object),
        "disease_name": pd.Series([], dtype=object),
        "chembl": pd.Series([], dtype=np.float64),
        "ot_genetics_portal": pd.Series([], dtype=np.float64),
        "phewas_catalog": pd.Series([], dtype=np.float64),
        "eva": pd.Series([], dtype=np.float64),
        "genomics_england": pd.Series([], dtype=np.float64),
        "gene2phenotype": pd.Series([], dtype=np.float64),
        "uniprot_literature": pd.Series([], dtype=np.float64),
        "uniprot_variants": pd.Series([], dtype=np.float64),
        "clingen": pd.Series([], dtype=np.float64),
        "orphanet": pd.Series([], dtype=np.float64),
        "impc": pd.Series([], dtype=np.float64),
        "europepmc": pd.Series([], dtype=np.float64),
        "expression_atlas": pd.Series([], dtype=np.float64),
        "crispr": pd.Series([], dtype=np.float64),
        "cancer_biomarkers": pd.Series([], dtype=np.float64),
        "slapenrich": pd.Series([], dtype=np.float64),
        "progeny": pd.Series([], dtype=np.float64),
        "reactome": pd.Series([], dtype=np.float64),
        "sysbio": pd.Series([], dtype=np.float64),
        "cancer_gene_census": pd.Series([], dtype=np.float64),
        "intogen": pd.Series([], dtype=np.float64),
        "eva_somatic": pd.Series([], dtype=np.float64),
    }
)

In [68]:
display(ds_table00)
display(ds_table00.shape)
display(type(ds_table00))

,disease_ID,disease_name,chembl,ot_genetics_portal,phewas_catalog,eva,genomics_england,gene2phenotype,uniprot_literature,uniprot_variants,...,expression_atlas,crispr,cancer_biomarkers,slapenrich,progeny,reactome,sysbio,cancer_gene_census,intogen,eva_somatic


(0, 24)

pandas.core.frame.DataFrame

In [69]:
ds_table = pd.concat([ds_table00, ds_table0]).fillna(
    {
        "chembl": 0,
        "ot_genetics_portal": 0,
        "phewas_catalog": 0,
        "eva": 0,
        "genomics_england": 0,
        "gene2phenotype": 0,
        "uniprot_literature": 0,
        "uniprot_variants": 0,
        "clingen": 0,
        "orphanet": 0,
        "impc": 0,
        "europepmc": 0,
        "expression_atlas": 0,
        "crispr": 0,
        "cancer_biomarkers": 0,
        "slapenrich": 0,
        "progeny": 0,
        "reactome": 0,
        "sysbio": 0,
        "cancer_gene_census": 0,
        "intogen": 0,
        "eva_somatic": 0,
    }
)

In [70]:
display(ds_table.head())
display(ds_table.shape)
display(type(ds_table))

,disease_ID,disease_name,chembl,ot_genetics_portal,phewas_catalog,eva,genomics_england,gene2phenotype,uniprot_literature,uniprot_variants,...,expression_atlas,crispr,cancer_biomarkers,slapenrich,progeny,reactome,sysbio,cancer_gene_census,intogen,eva_somatic
0,EFO_0000180,HIV-1 infection,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0
1,EFO_0000181,head and neck squamous cell carcinoma,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.330597,0.0,0.0,0.0,0.0,0.0,0.0
2,EFO_0000182,hepatocellular carcinoma,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0
3,EFO_0000198,myelodysplastic syndrome,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0
4,EFO_0000199,oral squamous cell carcinoma,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0


(245, 24)

pandas.core.frame.DataFrame

# evidence


In [71]:
variables03 = {
    "ensemblId": ensembl_gene_ok,
    "efoIds": overall_table.disease_ID.tolist(),
}

## literature


### europepmc


In [72]:
query_evidence_europepmc = """
    query target ($ensemblId: String!, $efoIds: [String!]!) {
        target (ensemblId: $ensemblId) {
            evidences (efoIds: $efoIds, datasourceIds: "europepmc", size: 10000) {
                count
                rows {
                    datatypeId
                    datasourceId
                    disease {
                        id
                        name
                    }
                    score
                    textMiningSentences {
                        dEnd
                        tEnd
                        dStart
                        tStart
                        section
                        text
                    }
                    literature
                }
            }
        }
    }
"""

In [73]:
r401 = requests.post(
    base_url, json={"query": query_evidence_europepmc, "variables": variables03}
)

In [74]:
res401 = json.loads(r401.text)

In [75]:
europepmctable = (
    pd.DataFrame(list(flatsplode(res401["data"]["target"]["evidences"])))
    .rename(
        columns={
            "rows.datatypeId": "datatype_ID",
            "rows.datasourceId": "datasource_ID",
            "rows.disease.id": "disease_ID",
            "rows.disease.name": "disease_name",
            "rows.score": "score",
            "rows.literature": "literature_ID",
            "rows.textMiningSentences.dEnd": "dEnd",
            "rows.textMiningSentences.tEnd": "tEnd",
            "rows.textMiningSentences.dStart": "dStart",
            "rows.textMiningSentences.tStart": "tStart",
            "rows.textMiningSentences.section": "section",
            "rows.textMiningSentences.text": "text",
        }
    )
    .filter(
        [
            "datatype_ID",
            "datasource_ID",
            "disease_ID",
            "disease_name",
            "score",
            "literature_ID",
            "section",
            "text",
            "tStart",
            "tEnd",
            "dStart",
            "dEnd",
        ]
    )
    .assign(
        literature_ID=lambda x: np.where(
            x.literature_ID.str.isdecimal(),
            "MED:" + x.literature_ID,
            x.literature_ID,
        )
    )
    .assign(
        literature_ID=lambda x: np.where(
            x.literature_ID.str.startswith("PMC"),
            "PMC:" + x.literature_ID,
            x.literature_ID,
        )
    )
    .assign(
        literature_ID=lambda x: np.where(
            x.literature_ID.str.startswith("PPR"),
            "PPR:" + x.literature_ID,
            x.literature_ID,
        )
    )
    .assign(
        literature_ID=lambda x: np.where(
            x.literature_ID.str.startswith("WO"),
            "PAT:" + x.literature_ID,
            x.literature_ID,
        )
    )
    .assign(
        literature_ID=lambda x: np.where(
            x.literature_ID.str.startswith("EP"),
            "PAT:" + x.literature_ID,
            x.literature_ID,
        )
    )
    .assign(
        text=lambda x: x.apply(
            lambda y: y.text[: y.tStart]
            + "<T>"
            + y.text[y.tStart : y.tEnd]
            + "</T>"
            + y.text[y.tEnd :],
            axis=1,
        )
    )
    .assign(dStart=lambda x: np.where(x.tStart < x.dStart, x.dStart + 7, x.dStart))
    .assign(dEnd=lambda x: np.where(x.tEnd < x.dEnd, x.dEnd + 7, x.dEnd))
    .assign(
        text=lambda x: x.apply(
            lambda y: y.text[: y.dStart]
            + "<D>"
            + y.text[y.dStart : y.dEnd]
            + "</D>"
            + y.text[y.dEnd :],
            axis=1,
        )
    )
    .filter(
        [
            "datatype_ID",
            "datasource_ID",
            "disease_ID",
            "disease_name",
            "score",
            "literature_ID",
            "section",
            "text",
        ]
    )
    .sort_values(["disease_ID", "score"], ascending=[True, False])
    .reset_index(drop=True)
)

In [76]:
display(europepmctable.head())
display(europepmctable.shape)
display(type(europepmctable))

,datatype_ID,datasource_ID,disease_ID,disease_name,score,literature_ID,section,text
0,literature,europepmc,EFO_0000180,HIV-1 infection,0.18,MED:34388205,discuss,Our monocyte methylation data in AHI further h...
1,literature,europepmc,EFO_0000180,HIV-1 infection,0.18,MED:34388205,discuss,Our monocyte methylation data in AHI further h...
2,literature,europepmc,EFO_0000180,HIV-1 infection,0.18,MED:34388205,discuss,Our monocyte methylation data in AHI further h...
3,literature,europepmc,EFO_0000180,HIV-1 infection,0.10,MED:26121689,results,"Before <D>HIV-1 infection</D>, we found a 60% ..."
4,literature,europepmc,EFO_0000180,HIV-1 infection,0.06,MED:33690734,discuss,"However, it remains unclear how m6A modificati..."


(1928, 8)

pandas.core.frame.DataFrame

In [77]:
europepmctable.text[6]

'Conclusions  CCR7, ETS1, RUNX3, CCR1, C3AR1, LAMB1, <T>IRF7</T>, LGALS3 and CDKN3 both are DEGs and regulated by DMRs in OSCC, which are quite associated with the progression of OSCC and the survival of <D>HNSCC</D> patients.'

#### europepmc api


In [78]:
def split_list(l, n):
    for idx in range(0, len(l), n):
        yield l[idx : idx + n]

In [79]:
lit_ID = europepmctable.literature_ID.unique().tolist()

In [80]:
len(lit_ID)

578

In [81]:
lit_ID_list = list(split_list(lit_ID, 8))

In [82]:
len(lit_ID_list)

73

In [83]:
lit_ID_list[0]

['MED:34388205',
 'MED:26121689',
 'MED:33690734',
 'MED:35418195',
 'PPR:PPR167926',
 'MED:33520407',
 'MED:19858727',
 'MED:31429854']

In [84]:
epmc_table00 = pd.DataFrame(
    {
        "source": pd.Series([], dtype=object),
        "extId": pd.Series([], dtype=object),
        "pmcid": pd.Series([], dtype=object),
        "annotations.exact": pd.Series([], dtype=object),
        "annotations.tags.name": pd.Series([], dtype=object),
        "annotations.tags.uri": pd.Series([], dtype=object),
        "annotations.id": pd.Series([], dtype=object),
        "annotations.type": pd.Series([], dtype=object),
        "annotations.section": pd.Series([], dtype=object),
        "annotations.provider": pd.Series([], dtype=object),
        "fullTextIdList": pd.Series([], dtype=object),
        "annotations.prefix": pd.Series([], dtype=object),
        "annotations.postfix": pd.Series([], dtype=object),
        "annotations.subType": pd.Series([], dtype=object),
    }
)

In [85]:
epmc_table = epmc_table00.copy()

epmc_url = "https://www.ebi.ac.uk/europepmc/annotations_api/annotationsByArticleIds"

for i in range(len(lit_ID_list)):
    epmc_r = requests.get(
        epmc_url, params={"articleIds": lit_ID_list[i], "format": "JSON"}
    )
    epmc_res = json.loads(epmc_r.text)

    epmc_table0 = epmc_table00.copy()

    for j in range(len(lit_ID_list[i])):
        try:
            epmc_table0 = pd.concat(
                [epmc_table0, pd.DataFrame(list(flatsplode(epmc_res[j])))]
            )
        except:
            pass

    epmc_table = pd.concat([epmc_table, epmc_table0])

    time.sleep(10)

In [86]:
display(epmc_table.head())
display(epmc_table.tail())
display(epmc_table.shape)
display(type(epmc_table))

,source,extId,pmcid,annotations.exact,annotations.tags.name,annotations.tags.uri,annotations.id,annotations.type,annotations.section,annotations.provider,fullTextIdList,annotations.prefix,annotations.postfix,annotations.subType,annotations.frequency
0,MED,34388205,PMC8386872,methylation,methylation,http://identifiers.org/go/GO:0032259,http://europepmc.org/article/MED/34388205#euro...,Gene Ontology,Title (http://purl.org/dc/elements/1.1/title),Europe PMC,PMC8386872,l-type specific DNA,profiles in blood d,NaN,NaN
1,MED,34388205,PMC8386872,HIV,HIV,http://identifiers.org/taxonomy/11676,http://europepmc.org/article/MED/34388205#euro...,Organisms,Title (http://purl.org/dc/elements/1.1/title),Europe PMC,PMC8386872,blood during acute,infection persists,NaN,NaN
2,MED,34388205,PMC8386872,infection,infection,http://linkedlifedata.com/resource/umls-concep...,http://europepmc.org/article/MED/34388205#euro...,Diseases,Title (http://purl.org/dc/elements/1.1/title),Europe PMC,PMC8386872,od during acute HIV,persists despite pr,NaN,NaN
3,MED,34388205,PMC8386872,HIV-1,HIV-1,http://identifiers.org/taxonomy/11676,http://europepmc.org/article/MED/34388205#euro...,Organisms,Abstract (http://purl.org/dc/terms/abstract),Europe PMC,PMC8386872,,disrupts the host epigenetic landscape,NaN,NaN
4,MED,34388205,PMC8386872,pathogenesis,pathogenesis,http://identifiers.org/go/GO:0009405,http://europepmc.org/article/MED/34388205#euro...,Gene Ontology,Abstract (http://purl.org/dc/terms/abstract),Europe PMC,PMC8386872,quences for disease,", viral persistence,",NaN,NaN


,source,extId,pmcid,annotations.exact,annotations.tags.name,annotations.tags.uri,annotations.id,annotations.type,annotations.section,annotations.provider,fullTextIdList,annotations.prefix,annotations.postfix,annotations.subType,annotations.frequency
36,MED,30713215,NaN,ISG15,ISG15,http://purl.uniprot.org/uniprot/O02741,http://europepmc.org/article/MED/30713215#euro...,Gene_Proteins,Abstract (http://purl.org/dc/terms/abstract),Europe PMC,NaN,"conclude that IRF7,",", MAS1, TMPRSS2, NPT",NaN,NaN
37,MED,30713215,NaN,MAS1,MAS1,http://purl.uniprot.org/uniprot/P04201,http://europepmc.org/article/MED/30713215#euro...,Gene_Proteins,Abstract (http://purl.org/dc/terms/abstract),Europe PMC,NaN,"e that IRF7, ISG15,",", TMPRSS2, NPTX1, XL",NaN,NaN
38,MED,30713215,NaN,TMPRSS2,TMPRSS2,http://purl.uniprot.org/uniprot/O15393,http://europepmc.org/article/MED/30713215#euro...,Gene_Proteins,Abstract (http://purl.org/dc/terms/abstract),Europe PMC,NaN,"IRF7, ISG15, MAS1,",", NPTX1, XLOC_016191",NaN,NaN
39,MED,30713215,NaN,NPTX1,NPTX1,http://purl.uniprot.org/uniprot/Q15818,http://europepmc.org/article/MED/30713215#euro...,Gene_Proteins,Abstract (http://purl.org/dc/terms/abstract),Europe PMC,NaN,"G15, MAS1, TMPRSS2,",", XLOC_016191, XLOC_",NaN,NaN
40,MED,30713215,NaN,pathogenesis,pathogenesis,http://identifiers.org/go/GO:0009405,http://europepmc.org/article/MED/30713215#euro...,Gene Ontology,Abstract (http://purl.org/dc/terms/abstract),Europe PMC,NaN,ontribute critical roles in the,of PD.,NaN,NaN


(334799, 15)

pandas.core.frame.DataFrame

In [87]:
epmc_table["annotations.type"].value_counts()

Gene_Proteins                         154933
Organisms                              43479
Diseases                               39253
Gene Ontology                          29037
Gene Disease Relationship              22754
Chemicals                              12573
Experimental Methods                   12052
COVoc                                   9011
Sequence                                2169
Accession Numbers                       2145
Cell Line                               1999
Cell                                    1641
Organ Tissue                            1587
Resources                                347
Clinical Drug                            313
Phenotype                                262
Gene Function                            207
Anatomy                                  195
State                                    192
Pathway                                  131
Gene Mutations                           105
Treatment                                 96
Molecule  

In [88]:
len(epmc_table.extId.drop_duplicates())

576

##### gene proteins


In [89]:
epmc_gene_proteins1 = (
    epmc_table.rename(
        columns={
            "annotations.tags.uri": "uniprot_url",
            "annotations.id": "annotations_id",
            "annotations.type": "annotations_type",
        }
    )
    .assign(literature_ID=lambda x: x.source + ":" + x.extId)
    .filter(["literature_ID", "annotations_type", "uniprot_url"])
    .query("annotations_type == 'Gene_Proteins'")
    .drop_duplicates()
    .assign(
        uniprot_ID=lambda x: x.uniprot_url.str.replace(
            "http://purl.uniprot.org/uniprot/", "", regex=False
        )
    )
)

In [90]:
display(epmc_gene_proteins1.head())
display(epmc_gene_proteins1.shape)
display(type(epmc_gene_proteins1))

,literature_ID,annotations_type,uniprot_url,uniprot_ID
16,MED:34388205,Gene_Proteins,http://purl.uniprot.org/uniprot/P29259,P29259
46,MED:34388205,Gene_Proteins,http://purl.uniprot.org/uniprot/Q9CJ45,Q9CJ45
74,MED:34388205,Gene_Proteins,http://purl.uniprot.org/uniprot/P51527,P51527
96,MED:34388205,Gene_Proteins,http://purl.uniprot.org/uniprot/P21926,P21926
103,MED:34388205,Gene_Proteins,http://purl.uniprot.org/uniprot/O75973,O75973


(26923, 4)

pandas.core.frame.DataFrame

In [91]:
epmc_gene_proteins2 = (
    epmc_table.rename(
        columns={
            "annotations.tags.uri": "uniprot_url",
            "annotations.id": "annotations_id",
            "annotations.type": "annotations_type",
        }
    )
    .assign(
        literature_ID=lambda x: x.annotations_id.str.replace(
            "http://europepmc.org/article/", "", regex=False
        )
    )
    .assign(literature_ID=lambda x: x.literature_ID.str.replace("#.*", "", regex=True))
    .assign(literature_ID=lambda x: x.literature_ID.str.replace("/", ":", regex=False))
    .filter(["literature_ID", "annotations_type", "uniprot_url"])
    .query("annotations_type == 'Gene_Proteins'")
    .drop_duplicates()
    .assign(
        uniprot_ID=lambda x: x.uniprot_url.str.replace(
            "http://purl.uniprot.org/uniprot/", "", regex=False
        )
    )
)

In [92]:
display(epmc_gene_proteins2.head())
display(epmc_gene_proteins2.shape)
display(type(epmc_gene_proteins2))

,literature_ID,annotations_type,uniprot_url,uniprot_ID
16,MED:34388205,Gene_Proteins,http://purl.uniprot.org/uniprot/P29259,P29259
43,PMC:PMC8386872,Gene_Proteins,http://purl.uniprot.org/uniprot/P29259,P29259
46,PMC:PMC8386872,Gene_Proteins,http://purl.uniprot.org/uniprot/Q9CJ45,Q9CJ45
74,PMC:PMC8386872,Gene_Proteins,http://purl.uniprot.org/uniprot/P51527,P51527
96,PMC:PMC8386872,Gene_Proteins,http://purl.uniprot.org/uniprot/P21926,P21926


(29048, 4)

pandas.core.frame.DataFrame

In [93]:
epmc_gene_proteins3 = (
    epmc_table.query("pmcid.notna()")
    .rename(
        columns={
            "annotations.tags.uri": "uniprot_url",
            "annotations.id": "annotations_id",
            "annotations.type": "annotations_type",
        }
    )
    .assign(literature_ID=lambda x: "PMC:" + x.pmcid)
    .filter(["literature_ID", "annotations_type", "uniprot_url"])
    .query("annotations_type == 'Gene_Proteins'")
    .drop_duplicates()
    .assign(
        uniprot_ID=lambda x: x.uniprot_url.str.replace(
            "http://purl.uniprot.org/uniprot/", "", regex=False
        )
    )
)

In [94]:
display(epmc_gene_proteins3.head())
display(epmc_gene_proteins3.shape)
display(type(epmc_gene_proteins3))

,literature_ID,annotations_type,uniprot_url,uniprot_ID
16,PMC:PMC8386872,Gene_Proteins,http://purl.uniprot.org/uniprot/P29259,P29259
46,PMC:PMC8386872,Gene_Proteins,http://purl.uniprot.org/uniprot/Q9CJ45,Q9CJ45
74,PMC:PMC8386872,Gene_Proteins,http://purl.uniprot.org/uniprot/P51527,P51527
96,PMC:PMC8386872,Gene_Proteins,http://purl.uniprot.org/uniprot/P21926,P21926
103,PMC:PMC8386872,Gene_Proteins,http://purl.uniprot.org/uniprot/O75973,O75973


(25834, 4)

pandas.core.frame.DataFrame

In [95]:
epmc_gene_proteins = (
    pd.concat([epmc_gene_proteins1, epmc_gene_proteins2, epmc_gene_proteins3])
    .drop_duplicates()
    .sort_values(["literature_ID", "uniprot_ID"])
    .reset_index(drop=True)
)

In [96]:
display(epmc_gene_proteins.head())
display(epmc_gene_proteins.shape)
display(type(epmc_gene_proteins))

,literature_ID,annotations_type,uniprot_url,uniprot_ID
0,MED:10692430,Gene_Proteins,http://purl.uniprot.org/uniprot/P28172,P28172
1,MED:10692430,Gene_Proteins,http://purl.uniprot.org/uniprot/P35849,P35849
2,MED:10692430,Gene_Proteins,http://purl.uniprot.org/uniprot/P42165,P42165
3,MED:10692430,Gene_Proteins,http://purl.uniprot.org/uniprot/P51527,P51527
4,MED:10692430,Gene_Proteins,http://purl.uniprot.org/uniprot/Q4JF28,Q4JF28


(52844, 4)

pandas.core.frame.DataFrame

In [97]:
len(epmc_gene_proteins.literature_ID.drop_duplicates())

1000

In [98]:
len(epmc_gene_proteins.uniprot_ID.drop_duplicates())

6199

In [99]:
epmc_gene_proteins.uniprot_ID.value_counts()

Q92985    938
P51527    836
P28172    751
P35849    548
Q4JF28    497
         ... 
Q69383      1
P01571      1
Q5R1P0      1
Q80YQ2      1
Q62714      1
Name: uniprot_ID, Length: 6199, dtype: int64

In [100]:
uniprot_url = (
    "https://rest.uniprot.org/uniprotkb/stream?format=json&query=%28%28gene%3A"
    + genetable_ok.symbol.tolist()[0]
    + "%29%29%20AND%20%28reviewed%3Atrue%29&fields=accession,id,gene_primary"
)

In [101]:
uniprot_list = requests.get(uniprot_url)

In [102]:
uniprot_table = pd.DataFrame(list(flatsplode(json.loads(uniprot_list.text))))

In [103]:
display(uniprot_table.head(10))
display(uniprot_table.shape)
display(type(uniprot_table))

,results.primaryAccession,results.uniProtkbId,results.genes.geneName.value
0,P70434,IRF7_MOUSE,Irf7
1,Q92985,IRF7_HUMAN,IRF7


(2, 3)

pandas.core.frame.DataFrame

In [104]:
epmc_gene_proteins_ok = (
    epmc_gene_proteins.query(
        "uniprot_ID == @uniprot_table['results.primaryAccession'].tolist()"
    )
    .sort_values(["literature_ID", "uniprot_ID"])
    .reset_index(drop=True)
)

In [105]:
len(epmc_gene_proteins_ok.literature_ID.drop_duplicates())

995

In [106]:
display(epmc_gene_proteins_ok.head())
display(epmc_gene_proteins_ok.shape)
display(type(epmc_gene_proteins_ok))

,literature_ID,annotations_type,uniprot_url,uniprot_ID
0,MED:10692430,Gene_Proteins,http://purl.uniprot.org/uniprot/Q92985,Q92985
1,MED:10799898,Gene_Proteins,http://purl.uniprot.org/uniprot/Q92985,Q92985
2,MED:10893229,Gene_Proteins,http://purl.uniprot.org/uniprot/Q92985,Q92985
3,MED:11070172,Gene_Proteins,http://purl.uniprot.org/uniprot/Q92985,Q92985
4,MED:11444873,Gene_Proteins,http://purl.uniprot.org/uniprot/Q92985,Q92985


(1130, 4)

pandas.core.frame.DataFrame

In [107]:
epmc_gene_proteins_ok.uniprot_ID.value_counts()

Q92985    938
P70434    192
Name: uniprot_ID, dtype: int64

In [108]:
list(
    set(epmc_gene_proteins.literature_ID.tolist())
    - set(epmc_gene_proteins_ok.literature_ID.tolist())
)

['MED:32881988',
 'MED:18971424',
 'PMC:PMC7494076',
 'MED:29453856',
 'MED:25510902']

In [109]:
europepmctable.query(
    "literature_ID != @epmc_gene_proteins_ok.literature_ID.tolist()"
).head()

,datatype_ID,datasource_ID,disease_ID,disease_name,score,literature_ID,section,text
844,literature,europepmc,EFO_0000763,viral disease,0.03,MED:18971424,abstract,Using <T>irf7</T>(-/-) and wild-type bone marr...
892,literature,europepmc,EFO_0000763,viral disease,0.02,MED:32881988,discuss,"For example, IFN-α expression is completely ab..."
1164,literature,europepmc,EFO_0007328,influenza,0.03,MED:29453856,abstract,"So far, however, IFN regulatory factor <T>(IRF..."
1202,literature,europepmc,EFO_1000680,mucous membrane pemphigoid,0.06,MED:25510902,abstract,al. who demonstrate increased expression of in...
1203,literature,europepmc,EFO_1000680,mucous membrane pemphigoid,0.06,MED:25510902,abstract,al. who demonstrate increased expression of in...


In [110]:
europepmctable_ok = europepmctable.assign(
    score=lambda x: np.where(
        np.isin(x.literature_ID, epmc_gene_proteins_ok.literature_ID.tolist()),
        x.score,
        0,
    )
)

In [111]:
display(europepmctable_ok.head())
display(europepmctable_ok.shape)
display(type(europepmctable_ok))

,datatype_ID,datasource_ID,disease_ID,disease_name,score,literature_ID,section,text
0,literature,europepmc,EFO_0000180,HIV-1 infection,0.18,MED:34388205,discuss,Our monocyte methylation data in AHI further h...
1,literature,europepmc,EFO_0000180,HIV-1 infection,0.18,MED:34388205,discuss,Our monocyte methylation data in AHI further h...
2,literature,europepmc,EFO_0000180,HIV-1 infection,0.18,MED:34388205,discuss,Our monocyte methylation data in AHI further h...
3,literature,europepmc,EFO_0000180,HIV-1 infection,0.10,MED:26121689,results,"Before <D>HIV-1 infection</D>, we found a 60% ..."
4,literature,europepmc,EFO_0000180,HIV-1 infection,0.06,MED:33690734,discuss,"However, it remains unclear how m6A modificati..."


(1928, 8)

pandas.core.frame.DataFrame

In [112]:
europepmctable_ok.query("score == 0").sort_values("literature_ID").head()

,datatype_ID,datasource_ID,disease_ID,disease_name,score,literature_ID,section,text
844,literature,europepmc,EFO_0000763,viral disease,0.0,MED:18971424,abstract,Using <T>irf7</T>(-/-) and wild-type bone marr...
1202,literature,europepmc,EFO_1000680,mucous membrane pemphigoid,0.0,MED:25510902,abstract,al. who demonstrate increased expression of in...
1203,literature,europepmc,EFO_1000680,mucous membrane pemphigoid,0.0,MED:25510902,abstract,al. who demonstrate increased expression of in...
1164,literature,europepmc,EFO_0007328,influenza,0.0,MED:29453856,abstract,"So far, however, IFN regulatory factor <T>(IRF..."
892,literature,europepmc,EFO_0000763,viral disease,0.0,MED:32881988,discuss,"For example, IFN-α expression is completely ab..."


In [113]:
europepmctable_ok.sort_values(
    ["disease_ID", "score"], ascending=[True, False]
).reset_index(drop=True).head()

,datatype_ID,datasource_ID,disease_ID,disease_name,score,literature_ID,section,text
0,literature,europepmc,EFO_0000180,HIV-1 infection,0.18,MED:34388205,discuss,Our monocyte methylation data in AHI further h...
1,literature,europepmc,EFO_0000180,HIV-1 infection,0.18,MED:34388205,discuss,Our monocyte methylation data in AHI further h...
2,literature,europepmc,EFO_0000180,HIV-1 infection,0.18,MED:34388205,discuss,Our monocyte methylation data in AHI further h...
3,literature,europepmc,EFO_0000180,HIV-1 infection,0.10,MED:26121689,results,"Before <D>HIV-1 infection</D>, we found a 60% ..."
4,literature,europepmc,EFO_0000180,HIV-1 infection,0.06,MED:33690734,discuss,"However, it remains unclear how m6A modificati..."


In [114]:
europepmctable_ok_short = europepmctable_ok.filter(
    [
        "datatype_ID",
        "datasource_ID",
        "disease_ID",
        "disease_name",
        "score",
        "literature_ID",
    ]
).drop_duplicates()

In [115]:
display(europepmctable_ok_short.head())
display(europepmctable_ok_short.shape)
display(type(europepmctable_ok_short))

,datatype_ID,datasource_ID,disease_ID,disease_name,score,literature_ID
0,literature,europepmc,EFO_0000180,HIV-1 infection,0.18,MED:34388205
3,literature,europepmc,EFO_0000180,HIV-1 infection,0.10,MED:26121689
4,literature,europepmc,EFO_0000180,HIV-1 infection,0.06,MED:33690734
5,literature,europepmc,EFO_0000181,head and neck squamous cell carcinoma,0.05,MED:35418195
6,literature,europepmc,EFO_0000181,head and neck squamous cell carcinoma,0.03,PPR:PPR167926


(950, 6)

pandas.core.frame.DataFrame

In [116]:
n = sym.symbols("n")

In [117]:
sym.summation(1 / (n**2), (n, 1, sym.oo))

pi**2/6

In [118]:
(0.18 + 0.10 / (2**2) + 0.06 / (3**2)) / (math.pi**2 / 6)

0.12867790322576897

##### disease


In [119]:
epmc_diseases1 = (
    epmc_table.rename(
        columns={
            "annotations.tags.uri": "umls_url",
            "annotations.id": "annotations_id",
            "annotations.type": "annotations_type",
        }
    )
    .assign(literature_ID=lambda x: x.source + ":" + x.extId)
    .filter(["literature_ID", "annotations_type", "umls_url"])
    .query("annotations_type == 'Diseases'")
    .drop_duplicates()
    .assign(
        umls_ID=lambda x: x.umls_url.str.replace(
            "http://linkedlifedata.com/resource/umls-concept/", "", regex=False
        )
    )
)

In [120]:
display(epmc_diseases1.head())
display(epmc_diseases1.shape)
display(type(epmc_diseases1))

,literature_ID,annotations_type,umls_url,umls_ID
2,MED:34388205,Diseases,http://linkedlifedata.com/resource/umls-concep...,C0009450
25,MED:34388205,Diseases,http://linkedlifedata.com/resource/umls-concep...,C0275518
118,MED:34388205,Diseases,http://linkedlifedata.com/resource/umls-concep...,C0042769
481,MED:34388205,Diseases,http://linkedlifedata.com/resource/umls-concep...,C0042749
518,MED:34388205,Diseases,http://linkedlifedata.com/resource/umls-concep...,C0275524


(7691, 4)

pandas.core.frame.DataFrame

In [121]:
epmc_diseases2 = (
    epmc_table.rename(
        columns={
            "annotations.tags.uri": "umls_url",
            "annotations.id": "annotations_id",
            "annotations.type": "annotations_type",
        }
    )
    .assign(
        literature_ID=lambda x: x.annotations_id.str.replace(
            "http://europepmc.org/article/", "", regex=False
        )
    )
    .assign(literature_ID=lambda x: x.literature_ID.str.replace("#.*", "", regex=True))
    .assign(literature_ID=lambda x: x.literature_ID.str.replace("/", ":", regex=False))
    .filter(["literature_ID", "annotations_type", "umls_url"])
    .query("annotations_type == 'Diseases'")
    .drop_duplicates()
    .assign(
        umls_ID=lambda x: x.umls_url.str.replace(
            "http://linkedlifedata.com/resource/umls-concept/", "", regex=False
        )
    )
)

In [122]:
display(epmc_diseases2.head())
display(epmc_diseases2.shape)
display(type(epmc_diseases2))

,literature_ID,annotations_type,umls_url,umls_ID
2,MED:34388205,Diseases,http://linkedlifedata.com/resource/umls-concep...,C0009450
25,MED:34388205,Diseases,http://linkedlifedata.com/resource/umls-concep...,C0275518
34,PMC:PMC8386872,Diseases,http://linkedlifedata.com/resource/umls-concep...,C0009450
118,PMC:PMC8386872,Diseases,http://linkedlifedata.com/resource/umls-concep...,C0042769
264,PMC:PMC8386872,Diseases,http://linkedlifedata.com/resource/umls-concep...,C0275518


(8721, 4)

pandas.core.frame.DataFrame

In [123]:
epmc_diseases3 = (
    epmc_table.query("pmcid.notna()")
    .rename(
        columns={
            "annotations.tags.uri": "umls_url",
            "annotations.id": "annotations_id",
            "annotations.type": "annotations_type",
        }
    )
    .assign(literature_ID=lambda x: "PMC:" + x.pmcid)
    .filter(["literature_ID", "annotations_type", "umls_url"])
    .query("annotations_type == 'Diseases'")
    .drop_duplicates()
    .assign(
        umls_ID=lambda x: x.umls_url.str.replace(
            "http://linkedlifedata.com/resource/umls-concept/", "", regex=False
        )
    )
)

In [124]:
display(epmc_diseases3.head())
display(epmc_diseases3.shape)
display(type(epmc_diseases3))

,literature_ID,annotations_type,umls_url,umls_ID
2,PMC:PMC8386872,Diseases,http://linkedlifedata.com/resource/umls-concep...,C0009450
25,PMC:PMC8386872,Diseases,http://linkedlifedata.com/resource/umls-concep...,C0275518
118,PMC:PMC8386872,Diseases,http://linkedlifedata.com/resource/umls-concep...,C0042769
481,PMC:PMC8386872,Diseases,http://linkedlifedata.com/resource/umls-concep...,C0042749
518,PMC:PMC8386872,Diseases,http://linkedlifedata.com/resource/umls-concep...,C0275524


(7280, 4)

pandas.core.frame.DataFrame

In [125]:
epmc_diseases = (
    pd.concat([epmc_diseases1, epmc_diseases2, epmc_diseases3])
    .drop_duplicates()
    .sort_values(["literature_ID", "umls_ID"])
    .reset_index(drop=True)
)

In [126]:
display(epmc_diseases.head())
display(epmc_diseases.shape)
display(type(epmc_diseases))

,literature_ID,annotations_type,umls_url,umls_ID
0,MED:10692430,Diseases,http://linkedlifedata.com/resource/umls-concep...,C0009450
1,MED:10692430,Diseases,http://linkedlifedata.com/resource/umls-concep...,C0042769
2,MED:10799898,Diseases,http://linkedlifedata.com/resource/umls-concep...,C0009450
3,MED:10893229,Diseases,http://linkedlifedata.com/resource/umls-concep...,C0009450
4,MED:11070172,Diseases,http://linkedlifedata.com/resource/umls-concep...,C0009450


(15015, 4)

pandas.core.frame.DataFrame

In [127]:
len(epmc_diseases.literature_ID.drop_duplicates())

996

In [128]:
len(epmc_diseases.umls_ID.drop_duplicates())

1424

In [129]:
epmc_diseases.umls_ID.value_counts()

C0009450    783
C0042769    593
C0006826    344
C0021400    280
C0027651    251
           ... 
C1736136      1
C0014173      1
C0406645      1
C0238421      1
C0031099      1
Name: umls_ID, Length: 1424, dtype: int64